In [18]:
# **Copyright (c) 2024 Saurabh Mangal. All rights reserved.**

# This repository and its contents are private and are the exclusive Unauthorized copying, distribution, or use of this repository or any portion thereof is strictly prohibited without the express written permission . 

# **Author:** Saurabh Mangal

In [2]:
import socket
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="us-central1"

UNIQUE_PREFIX = socket.gethostname()
UNIQUE_PREFIX = re.sub('[^A-Za-z0-9]+', '', UNIQUE_PREFIX)

BUCKET_NAME = f"{PROJECT_ID}-{UNIQUE_PREFIX}-{LOCATION}"

BUCKET_URI = f"gs://{BUCKET_NAME}"  # @param {type:"string"}

! gcloud config set project $PROJECT_ID
! gcloud storage buckets create {BUCKET_URI} --project={PROJECT_ID} --location={LOCATION}
! mkdir output

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

Updated property [core/project].
Creating gs://my-project-0004-346516-pytorch112kagglewbi-us-central1/...
ERROR: (gcloud.storage.buckets.create) HTTPError 409: Your previous request to create the named bucket succeeded and you already own it.


In [ ]:

import time

import vertexai
from vertexai.preview.tuning import sft

# TODO(developer): Update project
vertexai.init(project=PROJECT_ID, location="us-central1")

sft_tuning_job = sft.train(
    source_model="gemini-1.0-pro-002",
    train_dataset="gs://cloud-samples-data/ai-platform/generative_ai/sft_train_data.jsonl",
    # The following parameters are optional
    validation_dataset="gs://cloud-samples-data/ai-platform/generative_ai/sft_validation_data.jsonl",
    epochs=4,
    adapter_size=4,
    learning_rate_multiplier=1.0,
    tuned_model_display_name="tuned_gemini_pro",
)

# Polling for job completion
while not sft_tuning_job.has_ended:
    time.sleep(60)
    sft_tuning_job.refresh()

print(sft_tuning_job.tuned_model_name)
print(sft_tuning_job.tuned_model_endpoint_name)
print(sft_tuning_job.experiment)

Creating SupervisedTuningJob
SupervisedTuningJob created. Resource name: projects/255766800726/locations/us-central1/tuningJobs/3395930069992996864
To use this SupervisedTuningJob in another session:
tuning_job = sft.SupervisedTuningJob('projects/255766800726/locations/us-central1/tuningJobs/3395930069992996864')
View Tuning Job:
https://console.cloud.google.com/vertex-ai/generative/language/locations/us-central1/tuning/tuningJob/3395930069992996864?project=255766800726


projects/255766800726/locations/us-central1/models/6441300854837346304@1
projects/255766800726/locations/us-central1/endpoints/8342740188820144128


In [ ]:
import vertexai
from vertexai.preview.tuning import sft

# TODO(developer): Update project
vertexai.init(project=PROJECT_ID, location="us-central1")

responses = sft.SupervisedTuningJob.list()

for response in responses:
    print(response)

resource name: projects/255766800726/locations/us-central1/tuningJobs/3395930069992996864
resource name: projects/255766800726/locations/us-central1/tuningJobs/7319084177482055680
resource name: projects/255766800726/locations/us-central1/tuningJobs/7028039051563237376
resource name: projects/255766800726/locations/us-central1/tuningJobs/4587228791016783872
resource name: projects/255766800726/locations/us-central1/tuningJobs/1746864801029554176
resource name: projects/255766800726/locations/us-central1/tuningJobs/2051983675783905280


In [9]:
import vertexai
from vertexai.preview.tuning import sft

# TODO(developer): Update project, location
vertexai.init(project=PROJECT_ID, location=LOCATION)

tuning_job_id = "2051983675783905280"
response = sft.SupervisedTuningJob(
    f"projects/{PROJECT_ID}/locations/{LOCATION}/tuningJobs/{tuning_job_id}"
)

print(response)

resource name: projects/255766800726/locations/us-central1/tuningJobs/2051983675783905280


### Test the tuned model with a prompt

#### to get the name of end point, you can open the model in Gemini UI console and select your tuned model and then look at the code.

In [14]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models


def multiturn_generate_content():
  vertexai.init(project="255766800726", location="us-central1")
  model = GenerativeModel(
    "projects/255766800726/locations/us-central1/endpoints/8342740188820144128",
  )
  chat = model.start_chat()
  print(chat.send_message(
      ["""test"""],
      generation_config=generation_config,
      safety_settings=safety_settings
  ))


generation_config = {
    "max_output_tokens": 1024,
    "temperature": 1,
    "top_p": 1,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

multiturn_generate_content()



candidates {
  content {
    role: "model"
    parts {
      text: "I tested. I am testing this."
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.04493164271116257
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.059756580740213394
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.07779744267463684
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.04986590892076492
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.0427221953868866
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.03919513151049614
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.05252167582511902
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.05252167582511902
  }
}
usage

In [15]:
# from vertexai.preview.generative_models import GenerativeModel
# from vertexai.preview import tuning
# from vertexai.preview.tuning import sft

# TUNING_JOB_ID = "8342740188820144128"
# TUNING_JOB_REGION = LOCATION 

# sft_tuning_job = sft.SupervisedTuningJob(f"projects/{PROJECT_ID}/locations/{TUNING_JOB_REGION}/tuningJobs/{TUNING_JOB_ID}")

# content = "test"

# print("sft_tuning_job.tuned_model_endpoint_name",sft_tuning_job.tuned_model_endpoint_name)
# tuned_model = GenerativeModel(sft_tuning_job.tuned_model_endpoint_name)
# print(tuned_model.generate_content(content))